# 「シングルマシン環境におけるモデル並列訓練」

【原題】Single-Machine Model Parallel Best Practices

【原著】[Shen Li](https://mrshenli.github.io/)

【元URL】https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年11月14日

【チュトーリアル概要】

モデル並列は分散訓練のテクニックで広く使用されています。

1つ前のチュートリアル（日本語6_1）では、複数のGPU上でニューラルネットワークを訓練する際に、 [DataParallel](https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html) を使用する方法を紹介しました。

この機能は同一のモデルをすべてのGPU上に複製し、各GPUが入力データの異なる部分を利用します。

データ並列化は訓練プロセスを大幅に高速化出来ますが、モデルが大きすぎて単一のGPUに収まりきらないようなユースケースではうまく機能しません。

<br>

本チュートリアルでは、**モデル並列化**を使ってこの問題に対処する方法を説明します。

モデル並列化では、`DataParallel` のようにモデル全体を各GPU上に複製するのではなく、単一のモデルを異なる複数のGPU上に分割します。

（モデル`m`が10層を有しているとすれば、`DataParallel`の場合には、各GPUはこれら10層の複製を保持することになりますが、2つのGPU上でモデル並列化を行った場合には、各GPUが5層を管理します）

モデル並列のアイデアの基本は、モデル内の異なるサブネットワークをそれぞれ異なるデバイス上に配置し、配置に応じた `forward` メソッドを実装してデバイス間で中間出力を移動させることです。

モデルの一部だけが個々のデバイス上で動作するため、デバイスのセットとしてはより大きなモデルをまとめて提供することができます。

なお、本投稿では、巨大なモデルを構築し、限られた数のGPUにそれらのモデルを詰め込むことはせず、代わりにモデル並列のアイデアに焦点を当てます。

実世界のアプリケーションにモデル並列のアイデアを適用するかは読者の皆様に委ねます。

**注意**

モデルが複数サーバーを横断する、分散モデル並列訓練については、[分散RPCフレームワーク入門](https://pytorch.org/tutorials/intermediate/rpc_tutorial.html) のサンプル例や詳細を参照してください。

## 基本的な使用方法

まずは2つの線形層から構成される、簡単なトイモデルから始めましょう。

このモデルを2つのGPU上で実行するには、各線形層を異なるGPU上に配置し、配置した層に合うように入力と中間の出力をGPU上に移動するだけです。

In [ ]:
%matplotlib inline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = torch.nn.Linear(10, 10).to('cuda:0')
        self.relu = torch.nn.ReLU()
        self.net2 = torch.nn.Linear(10, 5).to('cuda:1')

    def forward(self, x):
        x = self.relu(self.net1(x.to('cuda:0')))
        return self.net2(x.to('cuda:1'))

上記の `ToyModel` は、適切なデバイス上に線形層とテンソルを配置する4つの `to(device)` の呼び出しを除いては、単一のGPU 上での実装方法に非常に似ている点に留意してください。

そして、モデルの実装で、通常から変更が必要になる部分も、この点のみです。




なお、モデルが単一のGPU上に存在する場合と同様、`backward()` と `torch.optim` は自動的に勾配の情報を管理してくれます。


ただし、損失関数を呼び出す際には、ラベルと出力が同じデバイスに存在する必要があります。

In [ ]:
model = ToyModel()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

optimizer.zero_grad()
outputs = model(torch.randn(20, 10))
labels = torch.randn(20, 5).to('cuda:1')
loss_fn(outputs, labels).backward()
optimizer.step()

# 日本語訳注：追記
# ToyModelの最後のforwardの最終層は'cuda:1'で処理されるので、labelsも'cuda:1'に集めています

## 既存モジュールへのモデル並列の適用方法

コードを数行変更するだけで、既存のシングルGPU用のモジュールをマルチGPUで実行することも可能です。

下記のコードでは、 `torchvision.models.resnet50()` を、2つのGPU上に処理できるように分解する方法を解説しています。



内容としては、既存の `ResNet` モジュールを継承し、モデルの構築中に2つのGPU上に層を分割しています。

そして、 `forward` メソッドをオーバーライドし、中間出力を適宜移動することで2つのサブネットワークを結合させています。

In [ ]:
from torchvision.models.resnet import ResNet, Bottleneck

num_classes = 1000


class ModelParallelResNet50(ResNet):
    def __init__(self, *args, **kwargs):
        super(ModelParallelResNet50, self).__init__(
            Bottleneck, [3, 4, 6, 3], num_classes=num_classes, *args, **kwargs)

        self.seq1 = nn.Sequential(
            self.conv1,
            self.bn1,
            self.relu,
            self.maxpool,

            self.layer1,
            self.layer2
        ).to('cuda:0')

        self.seq2 = nn.Sequential(
            self.layer3,
            self.layer4,
            self.avgpool,
        ).to('cuda:1')

        self.fc.to('cuda:1')

    def forward(self, x):
        x = self.seq2(self.seq1(x).to('cuda:1'))
        return self.fc(x.view(x.size(0), -1))

上記のような実装は、モデルが大きすぎて単一のGPUに収まらないケースにも対応できます。

ですが、既にお気付きかもしれませんが、上記では、モデルが1つのGPUに入る大きさの場合には、シングルGPUで実行する場合よりも、実行が遅くなります。

これは、どの時点においても2つのGPUの内1つのGPUしか稼働せず、もう一方は何もせずにただ存在しているアイドル状態のためです。

特に、`layer2` と `layer3` の間では、中間出力を `cuda:0` から `cuda:1` にコピーする必要があるため、性能がさらに低下します。

実行時間に関して定量的な観測をするために一つ実験を行ってみましょう。

この実験では、ランダムな入力とラベルを用いて、 `ModelParallelResNet50` と既存の `torchvision.models.resnet50()` の訓練をします。

訓練を行った後にモデルが何か意味のある予測を出力するわけではありませんが、実行時間に関する、定量的な証拠を求めることができます。

In [ ]:
import torchvision.models as models

num_batches = 3
batch_size = 120
image_w = 128
image_h = 128


def train(model):
    model.train(True)
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    one_hot_indices = torch.LongTensor(batch_size) \
                           .random_(0, num_classes) \
                           .view(batch_size, 1)

    for _ in range(num_batches):
        # ランダムな入力値とラベルを生成
        inputs = torch.randn(batch_size, 3, image_w, image_h)
        labels = torch.zeros(batch_size, num_classes) \
                      .scatter_(1, one_hot_indices, 1)

        # フォワードパスの実行
        optimizer.zero_grad()
        outputs = model(inputs.to('cuda:0'))

        # バックワードパスの実行
        labels = labels.to(outputs.device)
        loss_fn(outputs, labels).backward()
        optimizer.step()

上記の `train(model)` メソッドは、損失関数として `nn.MSELoss` を、最適化関数として`optim.SGD` を使用しています。

また、処理内容自体は、 `128 x 128` の画像に対する訓練を想定し、これらの画像が1つのミニバッチあたり120枚含まれます。

このようなミニバッチが3つあるデータセットです。

最後に `timeit` を使用してから `train(model)` メソッドを10回実行し、標準偏差とともに実行時間をプロットします。

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
import numpy as np
import timeit

num_repeat = 10

stmt = "train(model)"

setup = "model = ModelParallelResNet50()"
# globals引数は、Python3系でのみ利用可能です。
# Python2系では、以下の記述をしてください。
# import __builtin__
# __builtin__.__dict__.update(locals())
mp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
mp_mean, mp_std = np.mean(mp_run_times), np.std(mp_run_times)

setup = "import torchvision.models as models;" + \
        "model = models.resnet50(num_classes=num_classes).to('cuda:0')"
rn_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
rn_mean, rn_std = np.mean(rn_run_times), np.std(rn_run_times)


def plot(means, stds, labels, fig_name):
    fig, ax = plt.subplots()
    ax.bar(np.arange(len(means)), means, yerr=stds,
           align='center', alpha=0.5, ecolor='red', capsize=10, width=0.6)
    ax.set_ylabel('ResNet50 Execution Time (Second)')
    ax.set_xticks(np.arange(len(means)))
    ax.set_xticklabels(labels)
    ax.yaxis.grid(True)
    plt.tight_layout()
    plt.savefig(fig_name)
    plt.close(fig)


plot([mp_mean, rn_mean],
     [mp_std, rn_std],
     ['Model Parallel', 'Single GPU'],
     'mp_vs_rn.png')

<img src="https://pytorch.org/tutorials/_images/mp_vs_rn.png"></img>

結果は、モデル並列の実装モデルの実行時間は `4.02/3.75-1=7%` と、7%長い時間がかかり、シングルGPUの実装モデルよりも遅い結果となりました。

つまり、GPU間でテンソルをコピーするやり取りに、およそ7%のオーバーヘッドが発生していると結論付けることができます。



ですが、実行を通して2つの内の1つのGPUがアイドル状態であることが分かっているので、改善の余地はあります。

一つの選択肢は、ミニバッチのパイプラインを分割し、一つの分割されたパイプラインが第二のサブネットワークに到達したときに、次の分割されたパイプラインを第一のサブネットワークに供給できるようにすることです。


これにより、連続した2つの分割されたパイプラインを2つのGPU上で同時に実行できるようになります。

## 入力のパイプライン化による高速化

次の実験では、120枚の画像を含む各バッチをさらに20枚の画像毎に分割します。

なお、PyTorchではCUDAの操作を非同期で開始するため、並列性を実現するために、わざわざマルチスレッドを生成するための実装追加は必要ありません。

In [ ]:
class PipelineParallelResNet50(ModelParallelResNet50):
    def __init__(self, split_size=20, *args, **kwargs):
        super(PipelineParallelResNet50, self).__init__(*args, **kwargs)
        self.split_size = split_size

    def forward(self, x):
        splits = iter(x.split(self.split_size, dim=0))
        s_next = next(splits)
        s_prev = self.seq1(s_next).to('cuda:1')
        ret = []

        for s_next in splits:
            # A. s_prev は cuda:1 上で実行されます。
            s_prev = self.seq2(s_prev)
            ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

            # B. s_next はAと同時に cuda:0 上で実行されます。
            s_prev = self.seq1(s_next).to('cuda:1')

        s_prev = self.seq2(s_prev)
        ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

        return torch.cat(ret)


setup = "model = PipelineParallelResNet50()"
pp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
pp_mean, pp_std = np.mean(pp_run_times), np.std(pp_run_times)

plot([mp_mean, rn_mean, pp_mean],
     [mp_std, rn_std, pp_std],
     ['Model Parallel', 'Single GPU', 'Pipelining Model Parallel'],
     'mp_vs_rn_vs_pp.png')

デバイス間のテンソルのコピー操作は、コピー元のデバイスとコピー先のデバイスの間のストリーム上で同期化されることに気を付けてください。

もし複数のストリームを作成した場合には、コピー操作が適切に同期化されるようにする必要があります。




コピー操作が終了する前にコピー元のテンソルを書き込んだり、コピー先のテンソルを読み書きしたりすると、未定義の動作になることがあります。

上記の実装においては、コピー元のデバイス、コピー先のデバイスの双方でデフォルトのストリームを使用しているだけなので、追加で強制的に同期を行う必要はありません。

<img src="https://pytorch.org/tutorials/_images/mp_vs_rn_vs_pp.png"></img>

実験の結果から、並列モデルのResNet50への入力をパイプライン化することで、約 `3.75/2.51-1=49%` と、訓練プロセスが約50%高速化することがわかります。

2GPUなので、理想である100％の高速化（処理時間半分）からは、まだかけ離れています。



新しいパラメーターである `split_sizes` をパイプライン並列の実装に導入しましたが、このパラメーターが訓練時間全体にどのように影響を与えるかが不透明です。

直感的に表現すれば、`split_size` に小さい値を設定した場合は多くの小さなCUDAカーネルが起動しますが、大きい値を設定した場合は、最初と最後の分割の間で比較的長いアイドル時間が生じるようになります。

どちらも最適な状態ではありません。

<br>

しかし、この実験には、最適な `split_size` の設定値があるかもしれません。

異なる `split_size` の設定値をいくつか試して実験を行い、最適な設定値を探してみましょう。

In [ ]:
means = []
stds = []
split_sizes = [1, 3, 5, 8, 10, 12, 20, 40, 60]

for split_size in split_sizes:
    setup = "model = PipelineParallelResNet50(split_size=%d)" % split_size
    pp_run_times = timeit.repeat(
        stmt, setup, number=1, repeat=num_repeat, globals=globals())
    means.append(np.mean(pp_run_times))
    stds.append(np.std(pp_run_times))

fig, ax = plt.subplots()
ax.plot(split_sizes, means)
ax.errorbar(split_sizes, means, yerr=stds, ecolor='red', fmt='ro')
ax.set_ylabel('ResNet50 Execution Time (Second)')
ax.set_xlabel('Pipeline Split Size')
ax.set_xticks(split_sizes)
ax.yaxis.grid(True)
plt.tight_layout()
plt.savefig("split_size_tradeoff.png")
plt.close(fig)

<img src="https://pytorch.org/tutorials/_images/split_size_tradeoff.png"></img>


実験の結果、`split_size` には12を設定することで最速の訓練速度を実現可能であり、`3.75/2.43-1=54%` と、高速化が期待できることがわかりました。



しかし、訓練プロセスをさらに加速させる機会はまだあります。

例えば、現在 `cuda:0` 上のすべての操作はデフォルトのストリーム上で行われています。

これは、次の分割の演算が前の分割のコピー操作と重ならないで行われていることを意味します。

<br>

しかし、前の分割と次の分割のテンソルは異なっているため、ある分割の演算が他の分割のコピー処理と重なっても問題はありません。



このように、前後の分割の処理が一部重なるように処理を流すには、双方のGPU上で複数のストリームを使用する実装をし、異なるサブネットワークの構造で異なるストリーム管理の方策を採る必要があります。

なお、一般的なマルチストリームの手法のすべてが、今回のモデル並列のユースケースで動作するものではないので、これ以上は本チュートリアルでは取り扱いません。

**注意:**<br>
本チュートリアルではいくつかの性能指標を示しています。

掲載コードを自身のマシン上で実行する際には、異なる数値が表示されるかもしれませんが、これは実行結果がハードウェアやソフトウェアに依存しているためです。

自身の環境で最良のパフォーマンスを得る上での適切なアプローチは、まず曲線を生成して最適な分割サイズを把握し、その分割サイズをパイプラインの入力に使用することです。